In [13]:
import formulallm.formula as f
from pathlib import Path
from autogen.coding import CodeBlock, LocalCommandLineCodeExecutor
from autogen import ConversableAgent, UserProxyAgent

In [6]:
work_dir = Path("coding")
work_dir.mkdir(exist_ok=True)

executor = LocalCommandLineCodeExecutor(timeout = 60,
                                        work_dir = work_dir,
                                        functions = [f.load, f.solve, f.list, f.extract]
                                        )


In [9]:
local_llm_config={
    "config_list": [
        {
            "model": "NotRequired", # Loaded with LiteLLM command
            "api_key": "NotRequired", # Not needed
            "base_url": "http://0.0.0.0:4000"  # Your LiteLLM URL
        }
    ],
    "cache_seed": None # Turns off caching, useful for testing different models
}

In [10]:
file_path = "/home/zhang0311/FormulaLLMPY/examples/data/MappingExample.4ml"

(Compiled) MappingExample.4ml
0.59s.


In [15]:
executor_agent = ConversableAgent(name="code_executor",
                                  system_message = '''You are a specialized code executor.
                                  Your task is to first load the DSL model.
                                  Then, solve and complete the partial models.
                                  Next, extract the solution to the partial model and report whether the model is "broken".''',
                                  llm_config = local_llm_config,
                                  code_execution_config={"executor": executor},
                                  human_input_mode="NEVER",
                                  is_termination_msg = lambda msg: "Terminate" in msg["content"],
                                  default_auto_reply= "Please continue. If everything is done, reply 'TERMINATE'.",
                                  description = '''DSL Code Executor. 
                                                Solves and completes partial models in the loaded code, 
                                                extracts solutions, and reports whether the models are broken.'''
                                  )

In [14]:
userProxy = UserProxyAgent(name="user_proxy",
                           system_message= '''Let the code_executor to run the "solve" command
                                            to try to solve the partial models in the code loaded.
                                            Ask the code_executor if the model is "broken"(unsolvable) or not.
                                           ''',
                           code_execution_config = False,
                           llm_config = local_llm_config,
                           human_input_mode="ALWAYS")

In [ ]:
chat_res = userProxy.initiate_chat(
    recipient = executor_agent,
    message = f"Load the DSL model from the path({file_path}), then try to solve the model and report whether 
                the model is broken",
    summary_method = "reflection_with_llm",
    summary_prompt = "Summarize the conversation"
)